# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [77]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

da = pd.read_csv("nhanes_2015_2016.csv")

In [78]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [79]:
# enter your code here
filter_women = da.loc[(da.RIAGENDR == 2) & (da.RIDAGEYR >= 35) & (da.RIDAGEYR <= 50)] #first filter the women, age 35 & 50 from the da table 
dw = filter_women   #store this series into a simple variabe dw
#then seperate the married from unmarried
dw['marital_status'] = da.DMDMARTL.replace({1: 'married', 2: 'unmarried', 3: 'unmarried', 4: 'unmarried', 5: 'unmarried', 6: 'unmarried', 77: 'unmarried', 99: 'npnan'})     
#also seperate college from no college
dw['college_status'] = da.DMDEDUC2.replace({1: 'no_college', 2: 'no_college', 3: 'no_college', 4: 'no_college', 5: 'college', 7:'no_college', 9: 'npnan'})

#table to display the restricted values
dw = dw[['marital_status', 'college_status']].dropna() #droping all columns with empty values
pd.crosstab(dw.marital_status, dw.college_status)   #creates a table showing length of occupied columns

<ipython-input-79-6d807b591d52>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dw['marital_status'] = da.DMDMARTL.replace({1: 'married', 2: 'unmarried', 3: 'unmarried', 4: 'unmarried', 5: 'unmarried', 6: 'unmarried', 77: 'unmarried', 99: 'npnan'})
<ipython-input-79-6d807b591d52>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dw['college_status'] = da.DMDEDUC2.replace({1: 'no_college', 2: 'no_college', 3: 'no_college', 4: 'no_college', 5: 'college', 7:'no_college', 9: 'npnan'})


college_status,college,no_college
marital_status,,
married,162,287
unmarried,72,266


In [69]:
#calculate the proportion of women who have completed college
dz = dw.groupby(dw.marital_status).agg({"college_status": [lambda x: np.mean(x=="college"), np.size]})
dz.columns = ["Proportion", "Total_n"] # The default column names are unclear, so we replace them here
print(dz)

#proportion of married or not married with college (based on value above)
women_educated = sm.stats.proportion_confint(162, 162+72)  #college, samples
print( '\n' 'The proportion of educated women, married or unmarried is: {}' .format(women_educated))

#proportion of only married, with college or no college
women_married = sm.stats.proportion_confint(162, 162 + 287)
print('The proportion of married women, educated or not is given as: {}' .format(women_married))

#next is to calculate 95% confidence intervals for each of these proportions.
    #Confidence intervals are closely connected to standard errors.
    #standard error essentially tells us how far we should expect an estimate to fall from the truth, and it is an estimation of the standard       deviation. 
    #A confidence interval is an interval that under repeated sampling covers the truth a defined proportion of the time, and 
    #In most settings, this "coverage probability" is set to 95%.
    #A 95% confidence interval can be constructed as the interval consisting of all points that are within two (or 1.96) standard errors of         the point estimate.


p = dz.Proportion.married # Female proportion
n = dz.Total_n.married # Total number of females
se_married = np.sqrt(p * (1 - p) / n)
print( '\n' 'Standard error value for married women is {}' .format(se_married))

p = dz.Proportion.unmarried # Male proportion
n = dz["Total_n"].unmarried # Total number of males
se_unmarried = np.sqrt(p * (1 - p) / n)
print('Standard error value for unmarried women is {}' .format(se_unmarried))
print('\t' 'We can see that the standard errors for the estimated proportion of married and unmarried women from the population is very close')



                Proportion  Total_n
marital_status                     
married           0.360802      449
unmarried         0.213018      338

The proportion of educated women, married or unmarried is: (0.6331722212833798, 0.7514431633320048)
The proportion of married women, educated or not is given as: (0.31638193710753626, 0.4052216263668512)

Standard error value for married women is 0.02266360248455356
Standard error value for unmarried women is 0.022270605048202215
	We can see that the standard errors for the estimated proportion of married and unmarried women from the population is very close


__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

The standard error for a proportion is maximized when the true proportion is around 1/2, and gets smaller as the true proportion approaches either 0 or 1. The estimated unmarried women proportion is closer to 1/2 than the estimated married women proportion. also, the unmarried women sample size is smaller than the married women sample size. Both of these factors lead to the married standard error being larger than the unmarried standard error.

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

The take away from the above tabulated data is that more married women are educated than unmarried women even within the same age range of classification. Also, that confidence level is very connected with standard error.

## Question 2

Construct 95% confidence intervals for the proportion of smokers who are female, and for the proportion of smokers who are male.  Then construct a 95% confidence interval for the difference between these proportions.

In [88]:
# enter your code here
ds = da
ds["smokers"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})  # np.nan represents a missing value
ds["genders"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

ds = ds[["smokers", "genders"]].dropna()  # dropna drops cases where either variable is missing
pd.crosstab(ds.smokers, ds.genders)

dsmoke = ds.groupby(ds.genders).agg({"smokers": [lambda x: np.mean(x=="Yes"), np.size]})
dsmoke.columns = ["Proportion", "Total_n"] # The default column names are unclear, so we replace them here

p = dsmoke.Proportion.Female # Female proportion
n = dsmoke.Total_n.Female # Total number of females
se_female = np.sqrt(p * (1 - p) / n)
print('Standard error value for female smokers is {}' .format(se_female))

p = dsmoke.Proportion.Male # Male proportion
n = dsmoke["Total_n"].Male # Total number of males
se_male = np.sqrt(p * (1 - p) / n)
print('Standard error value for male smokers is {}' .format(se_male))


p = dsmoke.Proportion.Female # Female proportion
n = dsmoke.Total_n.Female # Total number of females
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print('\n' 'proportion of female smokers is given as {} and {} for the lower and upper case boundaries respectively' .format(lcb, ucb))

p = dsmoke.Proportion.Male # Male proportion
n = dsmoke.Total_n.Male # Total number of males
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print('proportion of male smokers is given as {} and {} or the lower and upper case boundaries respectively' .format(lcb, ucb))

Standard error value for female smokers is 0.008444152146214435
Standard error value for male smokers is 0.009526078653689868

proportion of female smokers is given as 0.288294683866098 and 0.32139576027925865 for the lower and upper case boundaries respectively
proportion of male smokers is given as 0.49458714955108174 and 0.531929377873546 or the lower and upper case boundaries respectively


__Q2a.__ Discuss why it may be relevant to report the proportions of smokers who are female and male, and contrast this to reporting the proportions of males and females who smoke.

This means that we are 95% sure that the proportion of smokers among the female population will be between 28% and 32%. 
anf for male: That we are 95% sure that the proportion of smokers among the male population will be between 49% and 53%.





__Q2b.__ How does the width of the confidence interval for the difference of the two proportions compare to the widths of the confidence intervals for each proportion separately?

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [ ]:
# enter your code here

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [ ]:
# enter your code here

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [ ]:
# enter code here

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?


__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [ ]:
# insert your code here

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

In [1]:
# insert your code here

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?